In [93]:
# from spotify_api import SpotifyData
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import Spotify
import yaml
import pandas as pd

def login():
    with open('config.yml', "r") as f:
        client = yaml.safe_load(f)
    client_id = client['client_id']
    client_secret= client['client_secret']
    sp = Spotify(client_credentials_manager=SpotifyClientCredentials(
        client_id = client_id,
        client_secret = client_secret
    ))
    return sp
def sc(sp, type='track',q='the%201975'):
    req_search = sp.search(q=q,type=type)
    track_id = req_search['tracks']['items'][0]['id']
    print(q,track_id)
    return track_id


In [37]:
# login to access spotify API
sp=login()

In [115]:
# billboard hot songs list
hits = pd.read_csv('../data/raw/billboard_hot_100_2012to2022.csv')

# Add track_id column
q_df = hits[['Title','Artist']].apply(lambda x: f"{x['Title']} {x['Artist']}", axis=1)
_track_id1 = q_df[:500].apply(lambda x: sc(sp,q=x))
_track_id2 = q_df[500:1000].apply(lambda x: sc(sp,q=x))
_track_id3 = q_df[1000:].apply(lambda x: sc(sp,q=x))
hits['track_id'] = pd.concat([_track_id1, _track_id2,_track_id3], ignore_index=True)

Somebody That I Used to Know Gotye feat Kimbra 0RtMRbaqeadQFvlYxcAczv
Call Me Maybe Carly Rae Jepsen 3TGRqZ0a2l1LRblBkJoaDx
We Are Young Fun feat Janelle Monáe 7a86XRg84qjasly9f6bPSD
Payphone Maroon 5 feat Wiz Khalifa 1UhGntCpnptxUsNjFrweVe
Lights Ellie Goulding 11ozIUBoXAgRTVWelDn4pL
Glad You Came The Wanted 5yDL13y5giogKs2fSNf7sj
Stronger (What Doesn't Kill You) Kelly Clarkson 6D60klaHqbCl9ySc8VcRss
We Found Love Rihanna feat Calvin Harris 2FYVldsDrKYzQebSvqHpEz
Starships Nicki Minaj 1oHNvJVbFkexQc0BpQp7Y4
What Makes You Beautiful One Direction 4cluDES4hQEUhmXj6TXkSo
Wild Ones Flo Rida feat Sia 6Zgff84ArpQosabjzUhvlP
Set Fire to the Rain Adele 73CMRj62VK8nUS4ezD2wvi
Sexy and I Know It LMFAO 0obBFrPYkSoBJbvHfUIhkv
Some Nights Fun 6t6oULCRS6hnI7rm0h5gwl
Wide Awake Katy Perry 15sc2nZlDsr1I3FWYAQOKT
Good Feeling Flo Rida 2LEF1A8DOZ9wRYikWgVlZ8
Whistle Flo Rida 3bC1ahPIYt1btJzSSEyyrF
One More Night Maroon 5 4XNrMwGx1SqP01sqkGTDmo
Drive By Train 0KAiuUOrLTIkzkpfpn9jb9
The Motto Drake feat 

In [133]:
# add popularity column
import time
def get_popularity(x):
    result = sp.track(x)
    # track['genres'] = result['album']['genres']
    # track['label'] = result['album']['label']
    time.sleep(0.5)
    return result['popularity']

hits['popularity'] = hits['track_id'].apply(lambda x: get_popularity(x))

In [ ]:
# add all audio features column
audio_dct_keys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
audio_dct={}
for audio_dct_key in audio_dct_keys:
    audio_dct[audio_dct_key] = []

def get_audioFeatures(track_ids):
    for idc, vl in enumerate(track_ids):
        result = sp.audio_features(track_ids[idc])
        for k,v in result[0].items():
            audio_dct[k].append(v)
    return audio_dct
get_audioFeatures(hits[:400]['track_id'].values)
get_audioFeatures(hits[400:900]['track_id'].values)
get_audioFeatures(hits[900:]['track_id'].values)

hits = pd.concat([hits,pd.DataFrame.from_dict(audio_dct)],axis=1)


In [217]:
hits[:5]

,Year,Rank,Title,Artist,track_id,popularity,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2012,1,Somebody That I Used to Know,Gotye feat Kimbra,0RtMRbaqeadQFvlYxcAczv,34,0.646,0.642,0,-6.190,...,0.0341,0.376,128.992,audio_features,0RtMRbaqeadQFvlYxcAczv,spotify:track:0RtMRbaqeadQFvlYxcAczv,https://api.spotify.com/v1/tracks/0RtMRbaqeadQ...,https://api.spotify.com/v1/audio-analysis/0RtM...,273493,4
1,2012,2,Call Me Maybe,Carly Rae Jepsen,3TGRqZ0a2l1LRblBkJoaDx,70,0.783,0.580,7,-6.548,...,0.1080,0.660,120.021,audio_features,3TGRqZ0a2l1LRblBkJoaDx,spotify:track:3TGRqZ0a2l1LRblBkJoaDx,https://api.spotify.com/v1/tracks/3TGRqZ0a2l1L...,https://api.spotify.com/v1/audio-analysis/3TGR...,193400,4
2,2012,3,We Are Young,Fun feat Janelle Monáe,7a86XRg84qjasly9f6bPSD,70,0.378,0.638,10,-5.576,...,0.0849,0.735,184.086,audio_features,7a86XRg84qjasly9f6bPSD,spotify:track:7a86XRg84qjasly9f6bPSD,https://api.spotify.com/v1/tracks/7a86XRg84qja...,https://api.spotify.com/v1/audio-analysis/7a86...,250627,4
3,2012,4,Payphone,Maroon 5 feat Wiz Khalifa,1UhGntCpnptxUsNjFrweVe,1,0.782,0.674,11,-9.124,...,0.0612,0.828,110.011,audio_features,1UhGntCpnptxUsNjFrweVe,spotify:track:1UhGntCpnptxUsNjFrweVe,https://api.spotify.com/v1/tracks/1UhGntCpnptx...,https://api.spotify.com/v1/audio-analysis/1UhG...,231423,4
4,2012,5,Lights,Ellie Goulding,11ozIUBoXAgRTVWelDn4pL,62,0.703,0.731,8,-6.283,...,0.1380,0.719,120.028,audio_features,11ozIUBoXAgRTVWelDn4pL,spotify:track:11ozIUBoXAgRTVWelDn4pL,https://api.spotify.com/v1/tracks/11ozIUBoXAgR...,https://api.spotify.com/v1/audio-analysis/11oz...,244267,4


In [226]:
hits.to_csv('../data/processed/audio_data_processed.csv')

TypeError: to_csv() got an unexpected keyword argument 'index_col'

In [237]:
hits = pd.read_csv('../data/processed/audio_data_processed.csv', parse_dates=['Year'])
hits.drop("Unnamed: 0", axis=1, inplace=True)
hits

,Year,Rank,Title,Artist,track_id,popularity,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2012-01-01,1,Somebody That I Used to Know,Gotye feat Kimbra,0RtMRbaqeadQFvlYxcAczv,34,0.646,0.642,0,-6.190,...,0.0341,0.376,128.992,audio_features,0RtMRbaqeadQFvlYxcAczv,spotify:track:0RtMRbaqeadQFvlYxcAczv,https://api.spotify.com/v1/tracks/0RtMRbaqeadQ...,https://api.spotify.com/v1/audio-analysis/0RtM...,273493,4
1,2012-01-01,2,Call Me Maybe,Carly Rae Jepsen,3TGRqZ0a2l1LRblBkJoaDx,70,0.783,0.580,7,-6.548,...,0.1080,0.660,120.021,audio_features,3TGRqZ0a2l1LRblBkJoaDx,spotify:track:3TGRqZ0a2l1LRblBkJoaDx,https://api.spotify.com/v1/tracks/3TGRqZ0a2l1L...,https://api.spotify.com/v1/audio-analysis/3TGR...,193400,4
2,2012-01-01,3,We Are Young,Fun feat Janelle Monáe,7a86XRg84qjasly9f6bPSD,70,0.378,0.638,10,-5.576,...,0.0849,0.735,184.086,audio_features,7a86XRg84qjasly9f6bPSD,spotify:track:7a86XRg84qjasly9f6bPSD,https://api.spotify.com/v1/tracks/7a86XRg84qja...,https://api.spotify.com/v1/audio-analysis/7a86...,250627,4
3,2012-01-01,4,Payphone,Maroon 5 feat Wiz Khalifa,1UhGntCpnptxUsNjFrweVe,1,0.782,0.674,11,-9.124,...,0.0612,0.828,110.011,audio_features,1UhGntCpnptxUsNjFrweVe,spotify:track:1UhGntCpnptxUsNjFrweVe,https://api.spotify.com/v1/tracks/1UhGntCpnptx...,https://api.spotify.com/v1/audio-analysis/1UhG...,231423,4
4,2012-01-01,5,Lights,Ellie Goulding,11ozIUBoXAgRTVWelDn4pL,62,0.703,0.731,8,-6.283,...,0.1380,0.719,120.028,audio_features,11ozIUBoXAgRTVWelDn4pL,spotify:track:11ozIUBoXAgRTVWelDn4pL,https://api.spotify.com/v1/tracks/11ozIUBoXAgR...,https://api.spotify.com/v1/audio-analysis/11oz...,244267,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,2022-01-01,96,Flower Shops,Ernest feat Morgan Wallen,0De9jFjJ4eRLl7Yww2eBw1,70,0.527,0.461,7,-5.908,...,0.0831,0.227,128.153,audio_features,0De9jFjJ4eRLl7Yww2eBw1,spotify:track:0De9jFjJ4eRLl7Yww2eBw1,https://api.spotify.com/v1/tracks/0De9jFjJ4eRL...,https://api.spotify.com/v1/audio-analysis/0De9...,214405,3
1096,2022-01-01,97,To the Moon,Jnr Choi and Sam Tompkins,3rF1pPjaDjWuIUCTXpMGIv,50,0.782,0.733,7,-5.112,...,0.1340,0.599,144.098,audio_features,3rF1pPjaDjWuIUCTXpMGIv,spotify:track:3rF1pPjaDjWuIUCTXpMGIv,https://api.spotify.com/v1/tracks/3rF1pPjaDjWu...,https://api.spotify.com/v1/audio-analysis/3rF1...,218880,4
1097,2022-01-01,98,Unholy,Sam Smith and Kim Petras,2nCTOlmH43oR4DqI6xlCpm,1,0.585,0.639,8,-2.788,...,0.2120,0.740,91.917,audio_features,2nCTOlmH43oR4DqI6xlCpm,spotify:track:2nCTOlmH43oR4DqI6xlCpm,https://api.spotify.com/v1/tracks/2nCTOlmH43oR...,https://api.spotify.com/v1/audio-analysis/2nCT...,192052,4
1098,2022-01-01,99,One Mississippi,Kane Brown,4FdPnT2cFrpWCmWZd7GXc3,60,0.471,0.846,0,-5.269,...,0.1450,0.539,100.089,audio_features,4FdPnT2cFrpWCmWZd7GXc3,spotify:track:4FdPnT2cFrpWCmWZd7GXc3,https://api.spotify.com/v1/tracks/4FdPnT2cFrpW...,https://api.spotify.com/v1/audio-analysis/4FdP...,213719,4


In [238]:
hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Year              1100 non-null   datetime64[ns]
 1   Rank              1100 non-null   int64         
 2   Title             1100 non-null   object        
 3   Artist            1100 non-null   object        
 4   track_id          1100 non-null   object        
 5   popularity        1100 non-null   int64         
 6   danceability      1100 non-null   float64       
 7   energy            1100 non-null   float64       
 8   key               1100 non-null   int64         
 9   loudness          1100 non-null   float64       
 10  mode              1100 non-null   int64         
 11  speechiness       1100 non-null   float64       
 12  acousticness      1100 non-null   float64       
 13  instrumentalness  1100 non-null   float64       
 14  liveness          1100 n

In [266]:
import altair as alt
import datetime
import pandas as pd
year_input=[datetime.date(2019,1,1),datetime.date(2020,1,1)]

hits['rank_bin'] = pd.cut(hits['Rank'],bins=10,labels=['1-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100'])


In [339]:

hits_c1 = hits[hits['Year'].isin(year_input)]
hits_c1_bin = hits_c1.groupby('rank_bin').mean().reset_index()
hits_c1_bin = hits_c1_bin[['rank_bin','energy','popularity','speechiness','liveness','valence']].set_index(['rank_bin','popularity']).stack().reset_index(name='value').rename(columns={'level_2':'features'})
hits_c1_bin

,rank_bin,popularity,features,value
0,1-10,74.25,energy,0.585650
1,1-10,74.25,speechiness,0.132105
2,1-10,74.25,liveness,0.139070
3,1-10,74.25,valence,0.536850
4,11-20,67.15,energy,0.612250
5,11-20,67.15,speechiness,0.103585
6,11-20,67.15,liveness,0.177615
7,11-20,67.15,valence,0.546050
8,21-30,72.15,energy,0.576100
9,21-30,72.15,speechiness,0.147435


In [340]:
hits_c1 = hits[hits['Year'].isin(year_input)]
hits_c1_bin = hits_c1.groupby('rank_bin').mean().reset_index()
hits_c1_bin = hits_c1_bin[['rank_bin','energy','popularity','speechiness','liveness','valence']].set_index(['rank_bin','popularity']).stack().reset_index(name='value').rename(columns={'level_2':'features'})

alt.Chart(hits_c1_bin).mark_bar().encode(
    x='features:N',
    y=alt.Y('value',scale=alt.Scale(zero=False)),
    column='rank_bin:N',
    color='features:N',
    opacity='popularity:Q'
)

alt.Chart(...)

In [333]:
_t

,rank_bin,popularity,level_2,value
0,1-10,74.25,energy,0.585650
1,1-10,74.25,speechiness,0.132105
2,11-20,67.15,energy,0.612250
3,11-20,67.15,speechiness,0.103585
4,21-30,72.15,energy,0.576100
5,21-30,72.15,speechiness,0.147435
6,31-40,69.50,energy,0.595850
7,31-40,69.50,speechiness,0.117900
8,41-50,65.45,energy,0.585150
9,41-50,65.45,speechiness,0.133065


In [265]:
pd.cut(hits['Rank'],bins=10,labels=['1-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100'])

0       (0.901, 10.9]
1       (0.901, 10.9]
2       (0.901, 10.9]
3       (0.901, 10.9]
4       (0.901, 10.9]
            ...      
1095    (90.1, 100.0]
1096    (90.1, 100.0]
1097    (90.1, 100.0]
1098    (90.1, 100.0]
1099    (90.1, 100.0]
Name: Rank, Length: 1100, dtype: category
Categories (10, interval[float64, right]): [(0.901, 10.9] < (10.9, 20.8] < (20.8, 30.7] < (30.7, 40.6] ... (60.4, 70.3] < (70.3, 80.2] < (80.2, 90.1] < (90.1, 100.0]]